## 3. Análise Exploratória com Dados Limpos  
📒 `2.1-rc-eda-cleaned.ipynb`

**Objetivo:** Explorar os dados tratados por meio de estatísticas descritivas e visualizações gráficas para identificar padrões, tendências e possíveis relações entre variáveis.

📊 **Atividades:**
- Relação entre Variáveis Númericas (scatterplots, heatmap)
- Relação entre Variáveis Categóricas e o Alvo (`price`)
- Insights Relevantes
- Next Steps
